In [19]:
#Khai báo thư viện cần thiết
import pandas as pd
import datetime as dt
import numpy as np

In [20]:
#đọc file dữ liệu đã thu thập được
df = pd.read_excel("updated_orders_vinamilk (1).xlsx")
df.head()

,Order ID,Customer ID,Customer Name,Phone Number,Email,Date of Birth,Address,Product,Quantity,Price,Total Amount,Transaction Date
0,G8F1C,KH004,Nguyễn Văn Mai,851338908,mai@gmail.com,1962-12-21,"280 Hoàng Đạo Thúy, Thanh Xuân, Hà Nội",Sữa Hạt Cao Đạm Vinamilk,6,35000,210000,2024-03-11 12:14:08
1,AK1VR,KH014,Đinh Thị Hường,853511615,huong@gmail.com,2004-06-20,"136 Hoàng Đạo Thúy, Thanh Xuân, Hà Nội",Thức Uống Sữa Sôcôla Lúa Mạch Vinamilk SuSu Túi,4,12000,48000,2024-05-30 13:51:55
2,SXMEC,KH002,Hà Kiều Trinh,343413164,trinh@gmail.com,1979-11-27,"187 Hoàng Đạo Thúy, Thanh Xuân, Hà Nội",Sữa Bột Dielac Alpha 4 - Cho Trẻ Từ 2 Đến 6 Tuổi,5,180000,900000,2024-02-01 15:05:35
3,EK8PK,KH006,Phạm Thị Hạnh,236483503,hanh@gmail.com,2002-01-26,"162 Hoàng Đạo Thúy, Thanh Xuân, Hà Nội",Thức Uống Sữa Sôcôla Lúa Mạch Vinamilk SuSu Chai,3,14000,42000,2024-02-23 19:17:44
4,UN5Z3,KH013,Phạm Quốc Huy,854238849,huy@gmail.com,1977-10-13,"186 Hoàng Đạo Thúy, Thanh Xuân, Hà Nội",Sữa Chua Uống Vinamilk SuSu Dâu Chai,1,15000,15000,2024-02-24 18:56:45


In [23]:
# chuyển 'Transaction Date' thành kiểu dữ liệu datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

In [40]:
# Tính toán giá trị 
df_recency = df.groupby(by='Customer ID', as_index=False)['Transaction Date'].max()
df_recency.columns = ['CustomerID', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply( lambda x: (recent_date - x).days)
df_recency.head()

,CustomerID,LastPurchaseDate,Recency
0,KH001,2024-04-02 21:17:06,58
1,KH002,2024-04-05 17:59:06,56
2,KH003,2024-05-17 11:23:20,14
3,KH004,2024-05-22 14:07:48,9
4,KH005,2024-04-11 21:35:23,49


In [41]:
# Tính toán chỉ số Frequency
frequency_df = df.drop_duplicates().groupby(by=['Customer ID'], as_index=False)['Transaction Date'].count()
frequency_df.columns = ['CustomerID', 'Frequency']
frequency_df.head()

,CustomerID,Frequency
0,KH001,6
1,KH002,6
2,KH003,9
3,KH004,5
4,KH005,4


In [42]:
# Tính toán chỉ số Monetary
monetary_df = df.groupby(by='Customer ID', as_index=False)['Total Amount'].sum()
monetary_df.columns = ['CustomerID', 'Monetary']
monetary_df.head()

,CustomerID,Monetary
0,KH001,1356000
1,KH002,5148000
2,KH003,2413000
3,KH004,682000
4,KH005,726000


In [43]:
# Assign Recency scores based on the defined intervals
def assign_recency_score(days):
    if days <= 7:
        return 5
    elif days <= 30:
        return 4
    elif days <= 90:
        return 3
    elif days <= 150:
        return 2
    elif days <= 365:
        return 1
    else:
        return 0  # Nếu thời gian mua hàng trước đó quá lâu (hơn 1 năm), có thể gán điểm là 0

df_recency['R_Score'] = df_recency['Recency'].apply(assign_recency_score)

# Display the result
print(df_recency.head())


  CustomerID    LastPurchaseDate  Recency  R_Score
0      KH001 2024-04-02 21:17:06       58        3
1      KH002 2024-04-05 17:59:06       56        3
2      KH003 2024-05-17 11:23:20       14        4
3      KH004 2024-05-22 14:07:48        9        4
4      KH005 2024-04-11 21:35:23       49        3


In [29]:
# Assign Frequency scores based on the defined intervals
def assign_frequency_score(frequency):
    if frequency >= 15:
        return 5
    elif frequency >= 12:
        return 4
    elif frequency >= 8:
        return 3
    elif frequency >= 4:
        return 2
    elif frequency >= 1:
        return 1
    else:
        return 0  # Nếu không có đơn hàng nào, có thể gán điểm là 0

df_frequency['F_Score'] = df_frequency['Frequency'].apply(assign_frequency_score)

# Display the result
print(df_frequency.head())


  Customer ID  Frequency  F_Score
0       KH001          6        2
1       KH002          6        2
2       KH003          9        3
3       KH004          5        2
4       KH005          4        2


In [35]:
def assign_monetary_score(monetary):
    if monetary >= 8000000:
        return 5
    elif monetary >= 5000000:
        return 4
    elif monetary >= 3000000:
        return 3
    elif monetary >= 1000000:
        return 2
    elif monetary > 0:
        return 1
    else:
        return 0  # Nếu không có chi tiêu nào, có thể gán điểm là 0

df_monetary['M_Score'] = df_monetary['Monetary'].apply(assign_monetary_score)
print(df_monetary.head())

  Customer ID  Monetary  M_Score
0       KH001   1356000        2
1       KH002   5148000        4
2       KH003   2413000        2
3       KH004    682000        1
4       KH005    726000        1


In [46]:
# Rename the column in df_frequency and df_monetary to match df_recency
df_frequency.rename(columns={'Customer ID': 'CustomerID'}, inplace=True)
df_monetary.rename(columns={'Customer ID': 'CustomerID'}, inplace=True)

# Now you can proceed with merging
rfm = df_recency[['CustomerID', 'R_Score']].merge(df_frequency[['CustomerID', 'F_Score']], on='CustomerID').merge(df_monetary[['CustomerID', 'M_Score']], on='CustomerID')

# Display the merged dataframe
print(rfm.head())


  CustomerID  R_Score  F_Score  M_Score
0      KH001        3        2        2
1      KH002        3        2        4
2      KH003        4        3        2
3      KH004        4        2        1
4      KH005        3        2        1


In [47]:
# Merge all necessary columns into rfm dataframe
rfm = df_recency[['CustomerID', 'LastPurchaseDate', 'Recency', 'R_Score']].merge(df_frequency[['CustomerID', 'Frequency', 'F_Score']], on='CustomerID').merge(df_monetary[['CustomerID', 'Monetary', 'M_Score']], on='CustomerID')

# Display the merged dataframe
print(rfm.head())


  CustomerID    LastPurchaseDate  Recency  R_Score  Frequency  F_Score  \
0      KH001 2024-04-02 21:17:06       58        3          6        2   
1      KH002 2024-04-05 17:59:06       56        3          6        2   
2      KH003 2024-05-17 11:23:20       14        4          9        3   
3      KH004 2024-05-22 14:07:48        9        4          5        2   
4      KH005 2024-04-11 21:35:23       49        3          4        2   

   Monetary  M_Score  
0   1356000        2  
1   5148000        4  
2   2413000        2  
3    682000        1  
4    726000        1  
